# CMIP6 Zarr Benchmarks

## Dataset Preprocessing

Benchmarks were generated for multiple copies of the CMIP6 daily data to understand the performance for different data pre-processing options. These copies were differentiated by:

* data file format (netCDF or Zarr), and,
* different chunking configurations

Details on each dataset configuration:

1. kerchunk + netCDF: A kerchunk reference file for NetCDF files stored on S3. 
2. Zarr stores with different chunking configurations and pyramids.
   * Chunked to optimize for time series analysis: 
       * latitude: 252, longitude: 252, time: 365.
       * This dataset has larger chunks, but more timesteps are loaded into each chunk.
   * Chunked to optimize for visualization at a single time step.
       * latitude: 600, longitude: 1440, time: 1.
       * This dataset has small chunks, but will likely not work well for time series generation.
   * Chunked to optimize for both time series and visualization:
       * latitude: 600, longitude: 1440, time: 29.
       * This dataset has larger chunks, but more timesteps are loaded into each chunk.
3. Zarr store with no coordinate chunking. At this time there is a known issue with pangeo-forge data generation where coordinates are chunked. This makes a significant impact on performance.

## Code Profiling Methodology

The time to generate a tile at zoom 0 to 11 was tested for each data store 10 times. The mean of these times is reported. Tests were run on the VEDA JupyterHub and the details can be reviewed in the [profile.ipynb notebook of the tile-benchmarking](https://github.com/developmentseed/tile-benchmarking/blob/main/profiling/profile.ipynb) repo.

The libraries used to generate image tiles are xarray for reading the Zarr metadata and rio_tiler's XarrayReader for reading data from the NetCDFs on S3.

Code from these libraries is copied into [tile-benchmarking](https://github.com/developmentseed/tile-benchmarking/blob/main/profiling/zarr_reader.py) to generate tiles and is replicated in the titiler-xarray API codebase. Maintaining a copy enabled full control to add timers to blocks of code and logs to understand where time was being spent. Specifically:

* `import s3fs; s3fs.core.setup_logging("DEBUG")` was used to debug calls to S3. This was used to understand that the most time is spent opening the dataset, which was impacted by open all the coordinate chunks.
* Timing code blocks also demonstrated that the most time, other than opening the dataset, was spent in reprojecting the data. Time to reproject the data is positively correlated with the chunk size, since the minimum amount of data that can be read from S3 is the size of the data chunk.

# Interpretation of the Results

Note: The spatial resolution is the same for all datasets but chunk size difference due to chunk shape.

* The best performance was for data that was not chunked spatially and only had 1 timestep per chunk.
* The performance was about the same for the kerchunk reference of this dataset. It is important to consider this is because the NetCDF data is chunked the same way. Even though 365 time steps (days) are stored in each NetCDF file, it is chunked by day.
* Chunking the data spatially likely impacts performance at low zoom levels. Notice how the data chunked into spatial chunks of 262x262 has the greatest difference between the time to tile at zoom 0 vs zoom 11.

# Summary

Data at the CMIP6 spatial resolution (600 x 1440), if chunked at 1 timestep per chunk, will work pretty well. But this does not tell us at what point we should chunk or pyramid the data. For that, we direct readers to a future notebook on when to chunk.


In [22]:
import pandas as pd
import hvplot
pd.options.plotting.backend = 'holoviews'
import warnings
warnings.filterwarnings("ignore")

git_url_path = "https://raw.githubusercontent.com/developmentseed/tile-benchmarking/main/profiling/results"
df = pd.read_csv(f"{git_url_path}/profiled_zarr_results.csv")

In [25]:
df[['collection_name', 'chunks', 'lat_resolution', 'lon_resolution', 'chunk_size_mb', 'number_coord_chunks', 'zoom 0', 'zoom 11']].sort_values('zoom 0')

,collection_name,chunks,lat_resolution,lon_resolution,chunk_size_mb,number_coord_chunks,zoom 0,zoom 11
2,600_1440_1_no-coord-chunks,"{'time': 1, 'lat': 600, 'lon': 1440}",0.25,0.25,3.295898,3,200.876,221.971
0,kerchunk,"{'time': 1, 'lat': 600, 'lon': 1440}",0.25,0.25,3.295898,3,219.537,184.691
3,600_1440_29,"{'time': 29, 'lat': 600, 'lon': 1440}",0.25,0.25,95.581055,27,699.789,652.922
4,365_262_262,"{'time': 365, 'lat': 262, 'lon': 262}",0.25,0.25,95.577469,9,1430.254,706.272
1,600_1440_1,"{'time': 1, 'lat': 600, 'lon': 1440}",0.25,0.25,3.295898,732,1690.967,1668.944


In [24]:
zooms = [f"zoom {zlev}" for zlev in range(12)]
zoom_df = df[['collection_name'] + zooms]
melted = pd.melt(zoom_df, id_vars=['collection_name'], var_name='zoom', value_name='time in ms')
melted.plot.scatter(x='zoom', y='time in ms', by='collection_name', width=1000, height=500)

:NdOverlay   [collection_name]
   :Scatter   [zoom]   (time in ms)

# Open questions and future work:

* A fast response is returned for data with a chunk size of 3MB. But if data must be fetched from multiple chunks to create a single tile how does this impact performance? If you store your data in many small chunks when (in zoom level and resolution) does this impact performance?
* More exploration of the pyramiding option is required, since the pyramid has all timesteps in each chunk and thus is not faster than data chunked optimally for visualitation
* How to balance testing and optimizing for time series generation.
